In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df_glu = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df_glu = thermo_df(df_glu)
df_glu.shape

In [ ]:
df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df = thermo_df(df)
df.shape
# plotly_zone(df)

# Cut 2 times

In [ ]:
ms_idxs = list()
for idx, row in dftop.iterrows():
    ms_idxs.append((row.Mass, idx))
len(ms_idxs)

In [ ]:
ms_idxs[0]

In [ ]:
from itertools import combinations

In [ ]:
sums = list()
idxs = list()
for comb in combinations(ms_idxs, 3):
    masssum = sum([x[0] for x in comb]) - 18.0106 * 2
    sums.append(masssum)
    idx = [x[1] for x in comb]
    idxs.append(idx)
#     break

# len(sums), len(idxs)
print('done')

In [ ]:
df_sums = pd.Series(sums)
df_idxs = pd.DataFrame(idxs)

In [ ]:
import pickle
with open('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/PaperData/df_sums.pkl', 'wb') as f:
    pickle.dump(df_sums, f)

with open('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/PaperData/df_idxs.pkl', 'wb') as f:
    pikcle.dump(df_idxs, f)

In [ ]:
# ms = 24252.31 #25334.57 #25005.52 #24267.31
# masses = [23904.24, 23599.19, 24233.28] # Asp
masses = [24287.35, 24460.38] # Ala, Val
# masses = [24460.38] # Val
ms = masses[0]
df_chosen = df_sums[(df_sums>=ms-0.05)&(df_sums<ms+0.05)]
len(df_idxs.loc[df_chosen.index])

In [ ]:
df_idxs_chosen = df_idxs.loc[df_chosen.index]
df_idxs_chosen.shape

In [ ]:
np_idxs = np.array(df_idxs_chosen)
np_uni_idxs = np.unique(np_idxs)

In [ ]:
df_tmp = dftop.loc[np_uni_idxs]
plotly_zone(df_tmp)
df_tmp.shape
df_tmp.to_excel('/Users/bryan/Downloads/cut2times.xlsx')

In [ ]:
df_phe_5p = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/GraphsOutput/Graphs_1_24581/S3_gap_fill_5p.xlsx')
df_phe_5p.shape

In [ ]:
full_mass = 21839.9673 #23088.1066 #23393.1479 #24581.3
df_fd_phe = computational_data_seperation(df_phe_5p, df, full_mass, ignore_endpoints=True)
# df_fd_phe = computational_data_seperation()
# plotly_basecalling(*df_fd_phe)
plotly_zones(*df_fd_phe, names=["5'dots", "mid dots"])

In [ ]:
bcr = base_calling_random(df_fd_phe[1])
plotly_basecalling(*bcr, y='RT', annotate=False)

In [ ]:
bcr[1]

# Homology Search & Group

In [ ]:
def homology_groups(df):
    """ given ~25k area dots, doing homology search
        return a list of group, each group represents a tRNA specy and its isoforms 
    """
    def create_graph(bcr):
        G = nx.Graph()
        for x in bcr[1]:
            G.add_edge(x[0], x[1], name=x[2])

        cc = list(nx.connected_components(G))
        subgraphs = [G.subgraph(x) for x in cc]
        subgraphs.sort(key=len, reverse=True)
        return subgraphs
    
    bcr = base_calling_random(df, homology=True)
    subgraphs = create_graph(bcr)
    subgraphs_edges = [ [(x[0], x[1], x[2].get('name')) for x in sg.edges(data=True)] for sg in subgraphs]
    subgraphs_edges = [sorted(sg_edges, key=lambda x: x[0]) for sg_edges in subgraphs_edges]
    subgraphs_nodes = [list(sg.nodes) for sg in subgraphs]
    
    node_edge_pairs = list()
    for group in range(len(subgraphs)):
        df_nodes = df[df.Mass.isin(subgraphs_nodes[group])]
        edges = subgraphs_edges[group]
        node_edge_pairs.append((df_nodes, edges))
    
    return node_edge_pairs

In [ ]:
groups = homology_groups(df_25k)
groups[0]
# plotly_basecalling(*groups[0], y='Vol', annotate=False)

In [ ]:
df_25k = df[(df.Mass > 2.3E4) & (df.Mass < 2.6E4) & (df.Vol > 5E5)]
bcr = base_calling_random(df_25k, homology=True)
# plotly_basecalling(*bcr, y='Vol')

In [ ]:
G = nx.Graph()
for x in bcr[1]:
    G.add_edge(x[0], x[1], name=x[2])

cc = list(nx.connected_components(G))
subgraphs = [G.subgraph(x) for x in cc]
subgraphs.sort(key=len, reverse=True)
len(subgraphs)

In [ ]:
subgraphs_edges = [ [(x[0], x[1], x[2].get('name')) for x in sg.edges(data=True)] for sg in subgraphs]
subgraphs_nodes = [list(sg.nodes) for sg in subgraphs]

subgraphs_edges = [sorted(sg_edges, key=lambda x: x[0]) for sg_edges in subgraphs_edges]

In [ ]:
idx = 1
group = idx - 1
df_nodes = df[df.Mass.isin(subgraphs_nodes[group])]
df_edges = subgraphs_edges[group]
plotly_basecalling(df_nodes, df_edges, y='Vol', annotate=False)
df_nodes.Mass, df_edges

In [ ]:
for idx, sg in enumerate(subgraphs):
    edges = list(sg.edges(data=True))
    edges = [(x[0], x[1], x[2].get('name')) for x in edges]
    edges.sort(key=lambda x:x[0], reverse=True)
    print(f'Group{idx+1}\n', edges)